In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pathlib
main_path= pathlib.Path('..')
print (main_path.resolve())
sys.path.insert(0, str(main_path.resolve()))

In [ ]:
import logging
logger= logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import IPython.core.display as display

In [ ]:
import stash_ai.config as stash_ai_config
from stash_ai.config import config
stash_ai_config.config_file= main_path.joinpath('config.json')
print(stash_ai_config.config_file.resolve())
stash_ai_config.load_config()
config.base_dir
from stash_ai.db import get_session

In [ ]:
from utils.custom_logging import get_logger
logger= get_logger("stash_ai.stash_scenes", True)

import gradio as gr
if gr.NO_RELOAD:
    from deepface import DeepFace
from stash_ai.config import config
from stash_ai.db import get_session
from stash_ai.model import Performer, Scene, Img, ImgFile, ImageAnalysis, DeepfaceFace, FaceStatus
from utils.performer import get_performer_stash_image, load_performer, get_unknown_performer_image
from utils.scene import load_scene, create_or_update_scene_from_stash, extract_scene_images, decord_scene
from utils.image import image_analysis, get_annotated_image_analysis_path, get_face_image_path, get_face_phash, hashes_are_similar, group_faces, load_image_analysis_from_imgFiles
from datetime import timedelta
import random
from PIL import Image
from tqdm import tqdm 
from typing import List, Dict
import pandas as pd

In [ ]:
from stash_ai.stash_scenes import *

In [ ]:
from utils.performer import get_performer_stash_image, load_performer, get_unknown_performer_image
import shutil

In [ ]:
scene_id= 1392
state_scene_stash= {}
state_scene_stash= handler_load_scene(scene_id, state_scene_stash)
html, performers_images, state_scene_stash, nb_images= update_scene_infos(state_scene_stash)

In [ ]:
display.HTML(html)
#print (performers_images)
#for im in performers_images:
#    display.Image(im)
#, performers_images, state_scene_stash, nb_images

In [ ]:
display.JSON(state_scene_stash)
#print (performers_images)
#for im in performers_images:
#    display.Image(im)
#, performers_images, state_scene_stash, nb_images

In [ ]:
number_deepface_ident_face_conf=0.9
number_ident_hash_tolerance=15

In [ ]:
logger.info(f"ident_faces detector {config.face_recognition_model} min_confidence face {number_deepface_ident_face_conf} hash tolerance {number_ident_hash_tolerance} model {config.face_identification_model}")
logger.debug(f"ident_faces {state_scene_stash}")
dataset_dir= config.data_dir.joinpath('dataset/performers')
logger.info(f"Dataset dir: {dataset_dir.resolve()}")
if not dataset_dir.exists():
    logger.critical(f"Dataset directory does not exists {dataset_dir.resolve()}")

In [ ]:
with get_session() as session:
    for group_name, faces_list in state_scene_stash.get('groups', {}).items():
        logger.info(f"Group {group_name}")
        frames_results= []
        paths= []
        for face_id in faces_list:
            face: DeepfaceFace= session.get(DeepfaceFace, face_id)
            df= DeepFace.find(get_face_image_path(face), db_path=dataset_dir.resolve(), detector_backend=config.face_recognition_model, model_name=config.face_identification_model, enforce_detection=False)
            frames_results.append(df[0])
        df= pd.concat(frames_results)
        perf_ids= []
        perf_names= []
        for path in df.identity:
            #relative_path= pathlib.Path(path).relative_to(config.data_dir)
            match_face_id= int(pathlib.Path(path).stem.split("_")[0])
            match_face: DeepfaceFace= session.get(DeepfaceFace, match_face_id)
            if match_face is None or match_face.performer is None:
                perf_ids.append(None)
                perf_names.append(None)
            else:
                perf_ids.append(match_face.performer.id)
                perf_names.append(match_face.performer.name)
        df['PerformerId'] = perf_ids
        df['PerformerName'] = perf_names
        df['score']= ((df.groupby('PerformerId')['PerformerId'].transform('size')/len(df))+1) * df.groupby('PerformerId')['threshold'].transform('max')
        s= df.groupby('PerformerId').agg("max").sort_values('score', ascending=False).head(3)
        s.reset_index()
        for index, row in s.iterrows():
            print(f"id: {index} {row['PerformerName']} {round(row['score'],3)}")


In [ ]:
df

In [ ]:
df.groupby('PerformerId').agg("max").sort_values('score', ascending=False)

In [ ]:
s= df[df['score'] > 0.65].groupby('PerformerId').agg("max").sort_values('score', ascending=False).head(20)

In [ ]:
s.reset_index()
for performer_id, row in s.iterrows():
    print(f"{performer_id}")

In [ ]:
s= df.groupby('PerformerId').agg("max").sort_values('score', ascending=False).head(3)

In [ ]:
s.reset_index()
for index, row in s.iterrows():
    print(f"id: {index} {row['PerformerName']} {round(row['score'],3)}")

In [ ]:
perf_ids= []
perf_names= []
with get_session() as session:
    for path in df.identity:
        #relative_path= pathlib.Path(path).relative_to(config.data_dir)
        match_face_id= int(pathlib.Path(path).stem.split("_")[0])
        match_face: DeepfaceFace= session.get(DeepfaceFace, match_face_id)
        if match_face is None or match_face.performer is None:
            perf_ids.append(None)
            perf_names.append(None)
        else:
            perf_ids.append(match_face.performer.id)
            perf_names.append(match_face.performer.name)
    df['PerformerId'] = perf_ids
    df['PerformerName'] = perf_names
    df['score']= ((df.groupby('PerformerId')['PerformerId'].transform('size')/len(df))+1) * df.groupby('PerformerId')['threshold'].transform('max')    
    

In [ ]:
grouped= df.groupby(['PerformerId', 'PerformerName'])

In [ ]:
df['score']= ((df.groupby('PerformerId')['PerformerId'].transform('size')/len(df))+1) * df.groupby('PerformerId')['threshold'].transform('max')

In [ ]:
grouped.agg(count=pd.NamedAgg(column="hash", aggfunc="count"), threshold=pd.NamedAgg(column="threshold", aggfunc="mean")).sort_values(["count", "threshold"], ascending=False)

In [ ]:
from PIL import Image
from deepface import DeepFace
import pandas as pd
import numpy as np
import pathlib
from matplotlib import pyplot as plt, image
import string
from dataclasses import dataclass, field
from typing import List, Dict
import cv2
import matplotlib.patches as patches
import json
from imagehash import phash
import math
%matplotlib inline 


In [ ]:
safechars = string.ascii_lowercase + string.ascii_uppercase + string.digits + '.-_'
def to_safechars(input: str) -> str:
    return ''.join([c for c in input if c in safechars])

In [ ]:
config.data_dir

In [ ]:
from stash_ai.model import Performer
from sqlalchemy import select
stmt = select(Performer)
with get_session() as session:
    results= session.execute(stmt)
    for o in results:
        print(o)

In [ ]:
from stash_ai.model import Performer

with get_session() as session:
    performer= session.get(Performer, 1)
    print(performer)


In [ ]:
scene_id= 1659
scene_id= 1
#scene_id= 241
scene= None
html= None
total_frames= 0
frames_per_seconds= 0
duration= 0
url= None
performers_images= []
html= ""
scene= config.stash_interface.find_scene(scene_id)
if scene:
    url= scene.get("paths",{}).get("stream")
    html= f"""
    <h1>{scene.get('title')}</h1>
    <p>{scene.get('details')}</p>
    """
    for file_info in scene.get('files'):
        frames_per_seconds= file_info.get('frame_rate', 0)
        duration= file_info.get('duration', 0)
        total_frames= math.ceil(file_info.get('duration', 0) * file_info.get('frame_rate', 0))
        html+=f"""
        <p>Filename {file_info.get('basename')} duration: {file_info.get('duration')} seconds. Frame per seconds : {file_info.get('frame_rate')}. Total frames : {total_frames}</p>
        """
    for performer_data in scene.get('performers', []):
        performer_id= performer_data.get('id')
        logger.info(f"fetch_scene_from_stash Performer {performer_id}")
        if performer_id:
            with get_session() as session:
                performer= load_performer(performer_id, session)
                performer_img= None
                
                if performer is not None:
                    performer_img= get_performer_stash_image(performer)
                    performer_text= f"{performer.name} [{performer.id}]"
                else:
                    performer_text= f"Not found [{performer_id}]"
                
                if performer_img is None:
                    performer_img= get_unknown_performer_image()
                
                performers_images.append((performer_img, performer_text))
                session.commit()
else:
    print("Unable to load scene")


In [ ]:
#print(json.dumps(scene, indent=2))
display.display(display.JSON(scene))

In [ ]:
print(f"scene_id {scene_id}, url {url}, duration {duration}, total_frames {total_frames}, frames_per_seconds {frames_per_seconds} performers_images len:{len(performers_images)}")
display.display(display.HTML(html))


In [ ]:
from tqdm import tqdm
import gradio as gr
import imagehash

In [ ]:
def hash_distance(left_hash, right_hash):
    """Compute the hamming distance between two hashes"""
    if len(left_hash) != len(right_hash):
        raise ValueError('Hamming distance requires two strings of equal length')

    return sum(map(lambda x: 0 if x[0] == x[1] else 1, zip(left_hash, right_hash)))


def hashes_are_similar(left_hash, right_hash, tolerance=6):
    """
    Return True if the hamming distance between
    the image hashes are less than the given tolerance.
    """
    return (left_hash - right_hash) <= tolerance
    return hash_distance(left_hash, right_hash) <= tolerance

In [ ]:
video_url= url
video_duration= duration
fps= frames_per_seconds
extract_n_images_per_seconds= 0.2
number_of_samples=10
progress= None
#def extract_video_images(video_url, video_duration: float, fps: float, extract_n_images_per_seconds: float, number_of_samples, progress= None):
logger.info(f"extract_images URL: {video_url} Img per second : {extract_n_images_per_seconds} FPS: {fps} ")
images= {}
nb_frames= math.floor(video_duration*fps)

if progress is not None:
    progress(0, desc="Extracting...", total=nb_frames, unit="frame")
else:
    p= tqdm(desc="Extracting...", total=nb_frames, unit="frame")
    progress= p.update

extract_directory= config.data_dir.joinpath('scene_extraction').joinpath(f"extracted_images_{scene_id}")
extract_directory_512= config.data_dir.joinpath('scene_extraction').joinpath(f"extracted_images_{scene_id}_512")
logger.info(f"extract_images: {extract_directory.resolve()}")

for d in [extract_directory , extract_directory_512]:
    if d.exists():
        shutil.rmtree(d)
    d.mkdir(parents=True)

if video_url is None:
    gr.Warning("Url is empty, could not extract.")
else:
    nb_extracted= 0
    frame= 0
    try:
        logger.info(f"extract_images Open VideoCapture {video_url}")
        cap= cv2.VideoCapture(video_url)
        success= True
        while success:
            success, image= cap.read()
            if success:
                #h= imagehash.dhash(Image.fromarray(image), hash_size=8)
                pImg= Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                h= imagehash.phash(pImg)
                frame+= 1
                if frame % 500 == 0:
                    logger.debug(f"Image {nb_extracted} frame {frame} of {nb_frames}")
                if progress is not None:
                    #progress(frame/nb_frames)
                    progress()
                # if frame == 500:
                #     break
                for oh in images.keys():
                    if hashes_are_similar(h, oh, 20):
                        break
                else:
                    #logger.info(f"frame {frame} hash {h} {type(h)}")
                    #image_path= extract_directory.joinpath(f"{nb_extracted:010}.jpg")
                    image_path= extract_directory.joinpath(f"{h}.jpg")
                    if nb_extracted % 10 == 0:
                        logger.info(f"{image_path.name} : {pImg.size}")
                    pImg.save(image_path)
                    pImg.thumbnail((512,512))
                    image_path= extract_directory_512.joinpath(f"{h}.jpg")
                    if nb_extracted % 10 == 0:
                        logger.debug(f"{image_path.name} thumbnail: {pImg.size}")
                    pImg.save(image_path)
                    
                    nb_extracted+= 1
                    images[h]= image_path
                    

    except Exception as e:
        logger.error(f"extract_images Error extraction {video_url} : {e!s}")
        logger.exception(e)
    logger.info(f"extract_images Extracted {nb_extracted} / {frame}")
print(f"{len(images)} images")